In [40]:
# multivariate output 1d cnn example
from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from numpy import loadtxt
lines = loadtxt("trial0_revised11.txt", delimiter=",", unpack=False, usecols=(4,7,10,13,16,19,22,25,28,31))
print(lines[0][0])


0.160804


In [41]:
len(lines)

24003

In [42]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	print(len(sequences))
	for i in range(len(sequences)):
		seq_x, seq_y = sequences[i, :7], sequences[i, 7:]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


In [43]:

# horizontally stack columns
#dataset = hstack((in_seq1, in_seq2, out_seq))
#print(dataset)
# choose a number of time steps
n_steps = 3


In [44]:
# convert into input/output
X, y = split_sequences(lines, n_steps)
#X=X[None,:,:]
print(X.shape)
print(y.shape)

24003
(24003, 7)
(24003, 3)


In [45]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[-1]
print('shape='+str(n_features))

shape=7


In [46]:
visible = Input(shape=(7, 1))
cnn = Conv1D(filters=64, kernel_size=2, activation='relu')(visible)
cnn = MaxPooling1D(pool_size=2)(cnn)
cnn = Flatten()(cnn)
cnn = Dense(50, activation='relu')(cnn)
# define output 1
output1 = Dense(1)(cnn)
# define output 2
output2 = Dense(1)(cnn)
# define output 3
output3 = Dense(1)(cnn)


In [47]:
# tie together
model = Model(inputs=visible, outputs=[output1, output2, output3])
model.compile(optimizer='adam', loss='mse')
print(model.summary())


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 7, 1)]       0           []                               
                                                                                                  
 conv1d_10 (Conv1D)             (None, 6, 64)        192         ['input_11[0][0]']               
                                                                                                  
 max_pooling1d_7 (MaxPooling1D)  (None, 3, 64)       0           ['conv1d_10[0][0]']              
                                                                                                  
 flatten_7 (Flatten)            (None, 192)          0           ['max_pooling1d_7[0][0]']        
                                                                                            

In [48]:

y1 = y[:, 0].reshape((y.shape[0], 1))
y2 = y[:, 1].reshape((y.shape[0], 1))
y3 = y[:, 2].reshape((y.shape[0], 1))
print(X.shape)

(24003, 7)


In [50]:

# fit model
print('fitting')
model.fit(X, [y1,y2,y3], epochs=2000, verbose=0)

fitting


In [51]:
lines2 = loadtxt("trial3_revised11.txt", delimiter=",", unpack=False, usecols=(4,7,10,13,16,19,22,25,28,31))

In [60]:
seq_x2, seq_y2 = lines2[100, :7], lines2[100, 7:]
x2=list()
x2.append(seq_x2)
x2=array(x2)
print(x2.shape)

yhat = model.predict(x2, verbose=0)
print(yhat)
print(seq_y2)

(1, 7)
[array([[-0.31246033]], dtype=float32), array([[-0.40525827]], dtype=float32), array([[-0.29952988]], dtype=float32)]
[ 0.151382 -0.02439  -0.064336]
